#### Installing Required Library

In [ ]:
!python3 -m pip install --upgrade langchain openai deeplake Tiktoken

#### Creating Your Own dataset

In [ ]:
import os
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ['OPENAI_API_KEY'] = getpass.getpass('open ai api key')
os.environ['ACTIVELOOP_TOKEN'] = getpass.getpass('activeloop token')
embeddings = OpenAIEmbeddings()

open ai api key··········
activeloop token··········


In [ ]:
!git clone https://github.com/twitter/the-algorithm # replace any repository of your choice


Cloning into 'the-algorithm'...
remote: Enumerating objects: 9142, done.
remote: Counting objects: 100% (2438/2438), done.
remote: Compressing objects: 100% (1662/1662), done.
remote: Total 9142 (delta 597), reused 2349 (delta 593), pack-reused 6704
Receiving objects: 100% (9142/9142), 7.67 MiB | 14.05 MiB/s, done.
Resolving deltas: 100% (2818/2818), done.


In [ ]:
import os
from langchain.document_loaders import TextLoader

root_dir = '/content/the-algorithm'
docs = []

for dirpath,dirname,filenames in os.walk(root_dir):
  for file in filenames:
        try:
            loader = TextLoader(os.path.join(dirpath,file),encoding = "utf-8")
            docs.extend(loader.load_and_split())

        except Exception as e:
            pass

#### Divide the loaded file into chunks

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap = 0)
texts = text_splitter.split_documents(docs)

#### Upload dataset into Activeloop

In [ ]:
username = 'vishalug3016'

db = DeepLake(
    dataset_path=f"hub://{username}/twitter-algorithm",
    embedding=embeddings,
)
db.add_documents(texts)

Deep Lake Dataset in hub://vishalug3016/twitter-algorithm already exists, loading from the storage


Creating 31310 embeddings in 63 batches of size 500::  51%|█████     | 32/63 [18:58<18:23, 35.59s/it]


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for text-embedding-ada-002 in organization org-4WvbxdPrEP0u1TuVPWHwoZwd on tokens per min (TPM): Limit 150000, Requested 158945. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
db = DeepLake(dataset_path=f"hub://vishalug3016/twitter-algorithm", read_only=True, embedding=embeddings)


Deep Lake Dataset in hub://vishalug3016/twitter-algorithm already exists, loading from the storage


In [ ]:
#Dataset(path='hub://vishalug3016/twitter-algorithm', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

In [ ]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = 'cos'
retriever.search_kwargs["fetch_k"] = 100
#retriever.search_kwargs["max_marginal_relevance_search_by_vector"] = True
retriever.search_kwargs["k"] = 10

In [ ]:
def filter(x):
    if 'com.google' in x['text'].data()['value']:
        return False
    metadata = x['metadata'].data()['value']
    return 'scala' in metadata['source'] or 'py' in metadata['source']

# Uncomment the following line to apply custom filtering
# retriever.search_kwargs['filter'] = filter



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model='gpt-3.5-turbo-0613') # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [ ]:
questions = [
    "What does favCountParams do?",
    "is it Likes + Bookmarks, or not clear from the code?",
    "What are the major negative modifiers that lower your linear ranking parameters?",
    # "How do you get assigned to SimClusters?",
    # "What is needed to migrate from one SimClusters to another SimClusters?",
    # "How much do I get boosted within my cluster?",
    # "How does Heavy ranker work. what are it’s main inputs?",
    # "How can one influence Heavy ranker?",
    # "why threads and long tweets do so well on the platform?",
    # "Are thread and long tweet creators building a following that reacts to only threads?",
    # "Do you need to follow different strategies to get most followers vs to get most likes and bookmarks per tweet?",
    # "Content meta data and how it impacts virality (e.g. ALT in images).",
    # "What are some unexpected fingerprints for spam factors?",
    # "Is there any difference between company verified checkmarks and blue verified individual checkmarks?",
]
chat_history = []

for question in questions:
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What does favCountParams do? 

**Answer**: favCountParams is a parameter used in the context of TweetFavoritesFetcher. It is of type ThriftLinearFeatureRankingParams and is used for ranking tweets based on the favorite count. The specific functionality and usage of favCountParams may depend on the implementation of TweetFavoritesFetcher. 

-> **Question**: is it Likes + Bookmarks, or not clear from the code? 

**Answer**: No, it is not clear from the given code whether favCountParams is the sum of Likes and Bookmarks. 

-> **Question**: What are the major negative modifiers that lower your linear ranking parameters? 

**Answer**: The major negative modifiers that lower the linear ranking parameters are:

1. No text hit demotion: This modifier is applied when a hit does not have any text associated with it.
2. URL only hit demotion: This modifier is applied when a hit only contains a URL and no other text.
3. Name only hit demotion: This modifier is applied when a hit o